In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
openrouter_api_key = os.getenv("OPEN_ROUTER_API_KEY")
os.environ['OPENWEATHER_API_KEY'] = os.getenv('OPENWEATHER_API_KEY')

In [2]:
model_client= OpenAIChatCompletionClient(
  model="gemini-1.5-flash-8b",
  api_key=api_key,
)

In [5]:
from langchain_tavily import TavilySearch
web_search = TavilySearch(max_results=3, api_key=tavily_api_key)

def web_search_tool(query: str) -> str:
    """Search the web for the given query and return the results."""
    results = web_search.invoke(query)
    return results

In [6]:
web_search_tool("What is the capital of France?")

{'query': 'What is the capital of France?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Paris',
   'title': 'Paris - Wikipedia',
   'content': 'Paris is the capital and largest city of France. With an estimated population of 2,048,472 in January 2025 in an area of more than 105 km2 (41 sq mi),',
   'score': 0.86024034,
   'raw_content': None},
  {'url': 'https://home.adelphi.edu/~ca19535/page%204.html',
   'title': 'Paris facts: the capital of France in history',
   'content': 'Paris, France   ## Paris facts: Paris, the **capital of France** Paris is the **capital of France**, Paris has 2.234 million inhabitants ## Paris facts: Paris history Republic, Paris has a rich 2000 year history. See details of Paris churches, including Notre ## Paris facts: Paris, a world city Paris is a world capital city of shopping french fashion brands. All of this turns Paris into a ## Paris facts: the capital of France in history Before

In [9]:
def weather_tool(city: str) -> str:
    """Get the current weather for the given city."""
    import requests
    api_key = os.getenv("OPENWEATHER_API_KEY")
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        return f"The current temperature in {city} is {data['main']['temp']}°C with {data['weather'][0]['description']}."
    else:
        return f"Could not retrieve weather data for {city}: {data.get('message', 'Unknown error')}"
    

weather_tool("India")

'The current temperature in India is 14.3°C with overcast clouds.'

In [20]:
from langchain_community.utilities import ArxivAPIWrapper
arxiv = ArxivAPIWrapper(top_k_results=3)
arxiv.run("RAG")

'Published: 2024-07-26\nTitle: Modular RAG: Transforming RAG Systems into LEGO-like Reconfigurable Frameworks\nAuthors: Yunfan Gao, Yun Xiong, Meng Wang, Haofen Wang\nSummary: Retrieval-augmented Generation (RAG) has markedly enhanced the capabilities\nof Large Language Models (LLMs) in tackling knowledge-intensive tasks. The\nincreasing demands of application scenarios have driven the evolution of RAG,\nleading to the integration of advanced retrievers, LLMs and other complementary\ntechnologies, which in turn has amplified the intricacy of RAG systems.\nHowever, the rapid advancements are outpacing the foundational RAG paradigm,\nwith many methods struggling to be unified under the process of\n"retrieve-then-generate". In this context, this paper examines the limitations\nof the existing RAG paradigm and introduces the modular RAG framework. By\ndecomposing complex RAG systems into independent modules and specialized\noperators, it facilitates a highly reconfigurable framework. Modul

In [21]:
def research_tool(query: str) -> str:
    """Search for research papers related to the query."""
    results = arxiv.run(query)
    return results

In [25]:
assistant = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    tools=[
        web_search_tool,
        weather_tool,
        research_tool,
    ],
    system_message="You are a helpful assistant that can answer questions and perform tasks using web_search_tool for web related information and use weather_tool weather information and for research paper realed use the research_tool to fetch the research papers and only provide the relevant content and make sure to use the provided tools wisely.",
)

In [23]:
task = "What is the current weather in Paris?"
async def main():
    response = await assistant.run(task=task)

    print(f"Agent Response: {response}")

await main()

Agent Response: messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the current weather in Paris?', type='TextMessage'), ToolCallRequestEvent(source='Assistant', models_usage=RequestUsage(prompt_tokens=125, completion_tokens=5), metadata={}, content=[FunctionCall(id='', arguments='{"city":"Paris"}', name='weather_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The current temperature in Paris is 20.01°C with scattered clouds.', name='weather_tool', call_id='', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='Assistant', models_usage=None, metadata={}, content='The current temperature in Paris is 20.01°C with scattered clouds.', type='ToolCallSummaryMessage')] stop_reason=None


In [26]:
task = "What is the current status of ICC Champions league?"
async def main():
    response = await assistant.run(task=task)

    print(f"Agent Response: {response}")

await main()

Agent Response: messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the current status of ICC Champions league?', type='TextMessage'), ToolCallRequestEvent(source='Assistant', models_usage=RequestUsage(prompt_tokens=127, completion_tokens=10), metadata={}, content=[FunctionCall(id='', arguments='{"query":"ICC Champions League status"}', name='web_search_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="{'query': 'ICC Champions League status', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.icc-cricket.com/tournaments/champions-trophy-2025/standings', 'title': 'Standings | ICC Champions Trophy, 2025', 'content': 'ICC STANDINGS PREDICTOR\\u200b\\u200b Click here to access the Official ICC tournament predictor. Find out what your team needs to do to make the Semi-Finals stage of the', 'score': 0.16178352, 'r

In [27]:
task = "Tell About the recent Research on RAG?"
async def main():
    response = await assistant.run(task=task)

    print(f"Agent Response: {response}")

await main()

Agent Response: messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Tell About the recent Research on RAG?', type='TextMessage'), ToolCallRequestEvent(source='Assistant', models_usage=RequestUsage(prompt_tokens=535, completion_tokens=8), metadata={}, content=[FunctionCall(id='', arguments='{"query":"Recent research on RAG"}', name='research_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="Published: 2025-06-26\nTitle: Response Quality Assessment for Retrieval-Augmented Generation via Conditional Conformal Factuality\nAuthors: Naihe Feng, Yi Sui, Shiyi Hou, Jesse C. Cresswell, Ga Wu\nSummary: Existing research on Retrieval-Augmented Generation (RAG) primarily focuses\non improving overall question-answering accuracy, often overlooking the quality\nof sub-claims within generated responses. Recent methods that attempt to\nimprove RAG trustworthiness, such as